# Chapter 7
## Ensemble Learning and Random Forests
_Enemble learning_ is essentially multiple machine learning models aggregating to produce a consensus answer. If you were to do this with decision trees, this would end up being a random forests.

Here, we will discuss popular methods such as bagging, boosting, stacking, and a few others.

## Voting Classifiers
Suppose you have a group of classifers that each acchieve about 80% accuracy. One simple way to get an even better classifer is to aggregate all of the results. This majority-vote classifer is called _hard voting_. This works most effectivly when the predictors are as independent as possible.

Here's a breif example of a voting classifer using three diverse classifers.

In [1]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

# TODO: Split into train and test sets
X, y = make_moons(n_samples=10000, noise=0.4)
X_train, X_test, y_train, y_test = train_test_split(X, y)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomF...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

An a quick look at all of the accuracys...

In [2]:
from sklearn.metrics import accuracy_score


for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8324
RandomForestClassifier 0.8404
SVC 0.86
VotingClassifier 0.8572


/home/marflo356/.local/share/virtualenvs/ML-Practice-JQdyjNbZ/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## Bagging and Pasting
One way to get a set of diverse classifiers is to do the method above and use very different algorithems.. or you can use the same algorithm on different random subsets of the traning set. When done with replacement, it is called _bagging_(bootstrap aggregating), when without replacement it's _pasting_.

Once all trained, you simplely aggregate the predictions. This is called _statistical mode_ for classification(hard voting), and average for regression.

The result is that compared to a single predictor, the bias is similar but the variance is lower. The predictors can be trained in parallel.

### Bagging and Pasting in Scikit-Learn
Here's 500 Decision Tree Classifiers with bagging!

In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1
)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

Bagging produces more variance than pasting, but overall less variance.

## Out-of-Bag Evaluation
With bagging, some instanes may not be sampled at all. Statistically speaking, about 63% of the training instances are sampled, leaving 37% of instances are not sampled, called _out-of-bag_(oob) instances.

Scikit let's you use this like sooo

In [5]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True
)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.8488

According to this, we should get about an 85 percent accuracy... Let's test this!

In [6]:
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.8344

## Random Patches and Random Subspaces
This does feature sampling instead of feature sampling. 'Tis usefull when working on high-dimensional data(images for example). Sampling both training and features is called _Random Patches method_. Keeping all of the instances buyt sampling features is called _Random Subspaces_ method.

## Random Forests
'Tis an ensemble of Decision Trees! Here it is as follows!

In [7]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

## Feature Importance
Decision trees have this intrisic property that all of the important features are towards the root of the tree, with the least important being towards the leaves. Scikit has feature importance built in to it to view it!

In [10]:
from sklearn.datasets import load_iris

iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.1129521046516854
sepal width (cm) 0.026382553067945822
petal length (cm) 0.40536981297814767
petal width (cm) 0.45529552930222106


## Boosting
